In [82]:
import numpy as np
import pandas as pd
from string import punctuation

In [137]:
def load_data():
    
    data_txt = open('data/shakespeare.txt')
    quatrain = []; volta = []; couplet = [];
    
    Lines = data_txt.readlines()
    count=0
    for i,line in enumerate(Lines):
        if any(char.isdigit() for char in line):
            for j in range(1,9):
                quatrain.append(Lines[i+j][:-1])
            for j in range(9,13):
                volta.append(Lines[i+j][:-1])
            for j in range(13,15):
                couplet.append(Lines[i+j][:-1])
    return quatrain,volta,couplet

def load_key():
    
    data_key = open('data/Syllable_dictionary.txt')
    Lines = data_key.readlines()
    
    key_list = [];
    
    for i,line in enumerate(Lines):
        split_str = line.strip(string.punctuation).split()
        if len(split_str) == 3:
            if len(split_str[1])==2:
                key_list.append([split_str[0],[int(split_str[2])],int(split_str[1][1])])
            elif len(split_str[2])==2:
                key_list.append([split_str[0],[int(split_str[1])],int(split_str[2][1])])
            else:
                key_list.append([split_str[0],[int(split_str[1]),int(split_str[2])],0])

        else:
            key_list.append([split_str[0],[int(split_str[1])],0])
    
    return key_list
    
    
    
def convert_to_nums(dataset,key_list):
    word_list = [i[0] for i in key_list]
    seq_list = []
    for i in range(0,len(dataset)):
        words = dataset[i].split()
        seq = []
        for j in range(0,len(words)):
            if words[j].lower() == "th'" or words[j].lower() == "t'":
                n = word_list.index(words[j].lower())
            else:
                n = word_list.index(words[j].lower().strip(string.punctuation))
            seq.append(n)
        seq_list.append(seq)
    
    return seq_list

def generate_sequences():
    #CALL THIS TO GENERATE THE 3 TRAINING DATASETS
    #OUTPUTS:
    #seq_list_quatrain: list of sequences corresponding to lines in the first 2 quatrains
    #seq_list_volta: list of sequences corresponding to lines in the volta
    #seq_list_couplet: list of sequences corresponding to lines in the couplet
    #key_list: list of lists containing information stored in Syllable_dictionary:
    #Each list corresponds to a word, with index 0 being the word, 
    #index 1 being a list containing possible numbers of syllables, and index 2 being the number of syllables if it's
    #an end word.
    
    quatrain,volta,couplet = load_data()
    key_list = load_key()
    seq_list_quatrain = convert_to_nums(quatrain,key_list)
    seq_list_volta = convert_to_nums(volta,key_list)
    seq_list_couplet = convert_to_nums(couplet,key_list)
    return seq_list_quatrain,seq_list_volta,seq_list_couplet,key_list
    
    
            
            

In [138]:
quatrain,volta,couplet = load_data()
key_list = load_key()
seq_list_quatrain = convert_to_nums(quatrain,key_list)

seq_list_quatrain,seq_list_volta,seq_list_couplet,key_list = generate_sequences()